In [ ]:
import torch
torch.cuda.is_available()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')

In [ ]:

import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
%matplotlib inline
pd.options.mode.chained_assignment = None
from sklearn.metrics import confusion_matrix, recall_score, precision_score, accuracy_score, f1_score,roc_auc_score
from ucm_model import UniversalChoiceModel as UCM
from ucm_model import *
import datetime
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import Adam, AdamW, SparseAdam, Adamax, Adagrad
from torch.nn.utils import clip_grad_norm_
from IPython.display import clear_output
import math
from sklearn.metrics import log_loss
torch.manual_seed(5)


In [ ]:
#data frame with the filtered topic labels and features that characterize each data entry:
nt = 50
file = open("gsdmm_topics.pkl",'rb')
df = pickle.load(file)
file.close()


In [ ]:
sns.set(font_scale=0.7)
df['text_tokens'].value_counts().plot(kind='bar', figsize=(9, 6), rot=0)
plt.xlabel("Topic labels", labelpad=14)
plt.ylabel("Number of topics", labelpad=14)
plt.title("Histogram showing frequency of topics in the dataset", y=1.02);

In [ ]:
#Tweet count for all the users considered
df['engaging_user_id'].value_counts()

In [ ]:
#Convert date to yyyy-mm-dd hh:mm:ss format

df['tweet_timestamp'] = df["tweet_timestamp"].apply(lambda x: datetime.datetime.fromtimestamp(x)) 

In [ ]:
t1 = df[(df["tweet_timestamp"] >= '2020-02-05 18:00:00') & (df["tweet_timestamp"] < '2020-02-06 06:00:00')]
t2 = df[(df["tweet_timestamp"] >= '2020-02-06 06:00:00') & (df["tweet_timestamp"] < '2020-02-06 18:00:00')]
t3 = df[(df["tweet_timestamp"] >= '2020-02-06 18:00:00') & (df["tweet_timestamp"] < '2020-02-07 06:00:00')]
t4 = df[(df["tweet_timestamp"] >= '2020-02-07 06:00:00') & (df["tweet_timestamp"] < '2020-02-07 18:00:00')]
t5 = df[(df["tweet_timestamp"] >= '2020-02-07 18:00:00') & (df["tweet_timestamp"] < '2020-02-08 06:00:00')]
t6 = df[(df["tweet_timestamp"] >= '2020-02-08 06:00:00') & (df["tweet_timestamp"] < '2020-02-08 18:00:00')]
t7 = df[(df["tweet_timestamp"] >= '2020-02-08 18:00:00') & (df["tweet_timestamp"] < '2020-02-09 06:00:00')]
t8 = df[(df["tweet_timestamp"] >= '2020-02-09 06:00:00') & (df["tweet_timestamp"] < '2020-02-09 18:00:00')]
t9 = df[(df["tweet_timestamp"] >= '2020-02-09 18:00:00') & (df["tweet_timestamp"] < '2020-02-10 06:00:00')]
t10 = df[(df["tweet_timestamp"] >= '2020-02-10 06:00:00') & (df["tweet_timestamp"] < '2020-02-10 18:00:00')]
t11 = df[(df["tweet_timestamp"] >= '2020-02-10 18:00:00') & (df["tweet_timestamp"] < '2020-02-11 06:00:00')]
t12 = df[(df["tweet_timestamp"] >= '2020-02-11 06:00:00') & (df["tweet_timestamp"] < '2020-02-11 18:00:00')]
t13 = df[(df["tweet_timestamp"] >= '2020-02-11 18:00:00') & (df["tweet_timestamp"] < '2020-02-12 06:00:00')]
t14 = df[(df["tweet_timestamp"] >= '2020-02-12 06:00:00') & (df["tweet_timestamp"] < '2020-02-12 18:00:00')]

# Each time period is 12 hrs
# Engagement frequency of user is from time periods t1 to t11 during training
# Recent history of user is considered from time periods t8 to t11 training 
# Training will be carrid out on time period t12
# Validation carried out on time period t13
# Testing will be carried out on time period t14


In [ ]:
#Engagement frequency for all available timeperiod
eng_frequency = df[df["tweet_timestamp"] < '2020-02-12 18:00:00']
eng_frequency['retweet'] = np.where(pd.notnull(eng_frequency['retweet_timestamp']), 1, 0)
columns = ['tweet_timestamp', 'tweet_id', 'reply_timestamp', "retweet_timestamp","retweet_with_comment_timestamp","like_timestamp"]
eng_frequency.drop(columns, axis=1,inplace=True)
engagement_history = eng_frequency.groupby(['engaging_user_id', 'text_tokens'])[['retweet']].agg('sum')
input_engagement_history = engagement_history.pivot_table(index='engaging_user_id', columns='text_tokens', values='retweet')
input_engagement_history.fillna(0,inplace = True)
history_frequency = pd.DataFrame(input_engagement_history.to_records())

In [ ]:
##Engagement frequency for testing time
eng_frequency_test = df[df["tweet_timestamp"] < '2020-02-12 6:00:00']
eng_frequency_test['retweet'] = np.where(pd.notnull(eng_frequency_test['retweet_timestamp']), 1, 0)
columns = ['tweet_timestamp', 'tweet_id', 'reply_timestamp', "retweet_timestamp","retweet_with_comment_timestamp","like_timestamp"]
eng_frequency_test.drop(columns, axis=1,inplace=True)
engagement_history_test = eng_frequency_test.groupby(['engaging_user_id', 'text_tokens'])[['retweet']].agg('sum')
input_engagement_history_test = engagement_history_test.pivot_table(index='engaging_user_id', columns='text_tokens', values='retweet')
input_engagement_history_test.fillna(0,inplace = True)
history_frequency_test = pd.DataFrame(input_engagement_history_test.to_records())
left_out_rows_xt = history_frequency[~history_frequency['engaging_user_id'].isin(history_frequency_test['engaging_user_id'])]
for col in left_out_rows_xt.columns:
    if col != 'engaging_user_id':
        left_out_rows_xt[col].values[:] = 0
final_history_xt = history_frequency_test.append(left_out_rows_xt)
final_history_xt = final_history_xt.sort_values('engaging_user_id')
history_frequency_test = final_history_xt.reset_index(drop=True)



In [ ]:
#Engagement frequency for training time
eng_frequency_train = df[df["tweet_timestamp"] < '2020-02-11 6:00:00']
eng_frequency_train['retweet'] = np.where(pd.notnull(eng_frequency_train['retweet_timestamp']), 1, 0)
columns = ['tweet_timestamp', 'tweet_id', 'reply_timestamp', "retweet_timestamp","retweet_with_comment_timestamp","like_timestamp"]
eng_frequency_train.drop(columns, axis=1,inplace=True)
engagement_history_train = eng_frequency_train.groupby(['engaging_user_id', 'text_tokens'])[['retweet']].agg('sum')
input_engagement_history_train = engagement_history_train.pivot_table(index='engaging_user_id', columns='text_tokens', values='retweet')
input_engagement_history_train.fillna(0,inplace = True)
history_frequency_train = pd.DataFrame(input_engagement_history_train.to_records())
left_out_rows_xt = history_frequency[~history_frequency['engaging_user_id'].isin(history_frequency_train['engaging_user_id'])]
for col in left_out_rows_xt.columns:
    if col != 'engaging_user_id':
        left_out_rows_xt[col].values[:] = 0
final_history_xt = history_frequency_train.append(left_out_rows_xt)
final_history_xt = final_history_xt.sort_values('engaging_user_id')
history_frequency_train = final_history_xt.reset_index(drop=True)

In [ ]:


columns = ['reply_timestamp','tweet_timestamp', 'tweet_id',"retweet_timestamp","retweet_with_comment_timestamp","like_timestamp"]


In [ ]:
def final_history(z):
    xt = z
    columns = ['reply_timestamp','tweet_timestamp', 'tweet_id',"retweet_timestamp","retweet_with_comment_timestamp","like_timestamp"]
    xt['retweet'] = np.where(pd.notnull(xt['retweet_timestamp']), 1, 0)
    xt.drop(columns, axis=1)
    x_t = xt.groupby(['engaging_user_id', 'text_tokens'])[['retweet']].agg('sum')
    xt_history = x_t.pivot_table(index='engaging_user_id', columns='text_tokens', values='retweet')
    xt_history.fillna(0,inplace = True)
    history_xt = pd.DataFrame(xt_history.to_records())
    for col in history_xt.columns:
      if col != 'engaging_user_id':
        history_xt.loc[history_xt[col] > 1, col] = 1

    left_out_rows_xt = history_frequency[~history_frequency['engaging_user_id'].isin(history_xt['engaging_user_id'])]
    for col in left_out_rows_xt.columns:
        if col != 'engaging_user_id':
            left_out_rows_xt[col].values[:] = 0
    final_history_xt = history_xt.append(left_out_rows_xt)
    final_history_xt = final_history_xt.sort_values('engaging_user_id')
    final_history_xt.reset_index(drop=True)
    final_history_xt = final_history_xt.fillna(0)
    return final_history_xt

In [ ]:
#dataframes with engagement histories  
from functools import reduce
recent_history = reduce(lambda x,y: pd.merge(x,y, on='engaging_user_id', how='outer'), [final_history(t8), final_history(t9), final_history(t10), final_history(t11)])
recent_history_test = reduce(lambda x,y: pd.merge(x,y, on='engaging_user_id', how='outer'), [final_history(t10), final_history(t11), final_history(t12), final_history(t13)])

In [ ]:
history_frequency = history_frequency.sort_values('engaging_user_id')
history_frequency_train = history_frequency_train.sort_values('engaging_user_id')

In [ ]:
def tweet_recommend(time):
    #time = df[(df["tweet_timestamp"] >= '2020-02-12 06:00:00') & (df["tweet_timestamp"] < '2020-02-12 18:00:00')]
    left = time[time['retweet_timestamp'] >= 0]
    time['retweet_timestamp'].fillna(0,inplace=True)

    left['retweet_timestamp'] = left["retweet_timestamp"].apply(lambda x: datetime.datetime.fromtimestamp(x))
    time = pd.concat([time,left]).drop_duplicates(keep=False)
    time['retweet_timestamp'] = time['tweet_timestamp']
    final = pd.concat([time,left])

    final = final.sort_values(['engaging_user_id', 'retweet_timestamp'], ascending=[True, False])
    final = final.reset_index(drop=True)

    columns = ['tweet_id',"reply_timestamp","retweet_with_comment_timestamp","like_timestamp"]
    final.drop(columns, axis=1,inplace=True)
    
    final = active_recommend(final)
    column = ['tweet_timestamp','retweet_timestamp']
    final.drop(column, axis=1,inplace=True)
    print (final['recommend'].value_counts())
    final_1 = final.groupby(['engaging_user_id', 'text_tokens'])[['recommend']].agg('sum')
    final_2 = final_1.pivot_table(index='engaging_user_id', columns='text_tokens', values='recommend')
    final_2.fillna(0,inplace = True)
    final_3 = pd.DataFrame(final_2.to_records())

    for col in final_3.columns:
      if col != 'engaging_user_id':
        final_3.loc[final_3[col] > 1, col] = 1


    left_out_rows_f = history_frequency[~history_frequency['engaging_user_id'].isin(final_3['engaging_user_id'])]
    for col in left_out_rows_f.columns:
        if col != 'engaging_user_id':
            left_out_rows_f[col].values[:] = 0
    final_4 = final_3.append(left_out_rows_f)
    recommend_test = final_4.sort_values('engaging_user_id')
    recommend_test.reset_index(drop=True, inplace=True)
    return(recommend_test)

    
    

In [ ]:
# We consider the positive examples for topic recommendations from all the tweets posted during the active states and vice versa. 
# An active state is defined as a period between when a tweet is published and engaged


#Function action_recommend for determining the active states
def active_recommend(x):
    g = 3
    for index, row in x.iterrows():
      p = 0  
      if g != 10 :
        a = row['engaging_user_id']
        b = row['tweet_timestamp']
        c = row['retweet_timestamp']
        p = 0
        if b != c :
          g = 10
      if (row['tweet_timestamp'] != row['retweet_timestamp']):
          p = 1
          d = row['engaging_user_id']
          e = row['tweet_timestamp']
          f = row['retweet_timestamp']
      elif row['tweet_timestamp'] == row['retweet_timestamp']:
          if row['engaging_user_id'] == a:
            p = 0
          elif row['engaging_user_id'] == d:
            if (row['tweet_timestamp'] >= e) & (row['tweet_timestamp'] <= f):
              p = 1
            else:
              p = 0
      x.loc[index,'recommend'] = p
    return x

In [ ]:
recommend_test = tweet_recommend(df[(df["tweet_timestamp"] >= '2020-02-12 06:00:00') & (df["tweet_timestamp"] < '2020-02-12 18:00:00')])
recommend = tweet_recommend(df[(df["tweet_timestamp"] >= '2020-02-11 6:00:00') & (df["tweet_timestamp"] < '2020-02-11 18:00:00')])

In [ ]:
X_recent_2 = recent_history.drop('engaging_user_id', axis=1).values
X_recent_1 = X_recent_2.reshape(-1,nt,4)

X_recent_2_test = recent_history_test.drop('engaging_user_id', axis=1).values
X_recent_1_test = X_recent_2_test.reshape(-1,nt,4)



X_freq_1 = history_frequency_test.drop('engaging_user_id', axis=1).values
X_freq_1_train = history_frequency_train.drop('engaging_user_id', axis=1).values


X_recommend_1 = recommend.drop('engaging_user_id', axis=1).values
X_recommend_t = recommend_test.drop('engaging_user_id', axis=1).values

y_1 = final_history(t12).drop('engaging_user_id', axis=1).values

y_t = final_history(t14).drop('engaging_user_id', axis=1).values

#INPUTS



In [ ]:
X_recent = torch.FloatTensor(X_recent_1)              #Recent Engagement History tensor input for training
X_freq = torch.FloatTensor(X_freq_1_train)            #Engagement Frequency tensor input for training
X_recommend = torch.FloatTensor(X_recommend_1)        #Topic Recommendation tensor input for training
y = torch.FloatTensor(y_1)                            #Engagement output tensor input for training


X_recent_test = torch.FloatTensor(X_recent_1_test)    #Recent Engagement History tensor input for testing
X_freq_test = torch.FloatTensor(X_freq_1)             #Engagement Frequency tensor input for testing
X_recommend_test = torch.FloatTensor(X_recommend_t)   #Topic Recommendation tensor input for testing
Y_test = torch.FloatTensor(y_t)                       #Engagement output tensor input for testing

In [ ]:
X_freq = F.normalize(X_freq, p=2, dim=1)              #Normalized Engagement Frequency tensor input for training
X_freq_test = F.normalize(X_freq_test, p=2, dim=1)    #Normalized Engagement Frequency tensor input for testing

In [ ]:
filters = 20
L = 10
class DNN_f(nn.Module):
    def __init__(self):
        super().__init__()
        #frequency input
        self.fc1 = nn.Linear(in_features=nt, out_features=L)

        #history input 32 * 50 * 4
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = filters, kernel_size = (1,4) ,stride = 1)
        self.leaky = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels = filters, out_channels = filters, kernel_size = (nt-L+1,1) ,stride = 1)
        self.conv3 = nn.ConvTranspose2d(in_channels = filters, out_channels = filters, kernel_size =(nt-L+1,1), stride =1)


        #recommend input
        self.fc3 = nn.Linear(in_features=nt, out_features=L)
        
        #final linear
        self.conv4 = nn.Conv2d(in_channels = 1, out_channels=1, kernel_size=(1, 5+2*(filters)), stride = 1) 
        
        self.fc4 = nn.Linear(in_features=5+2*(filters), out_features=1)
        

 
    def forward(self, x, y, z, a):
        x1 = self.fc1(x)      ### EH
        x2 = F.linear(x1, self.fc1.weight.t())  #### EH BAR

        
        y_ = y.view(-1,1,nt,4)
        y1 = self.conv1(y_)
        y1 = self.leaky(y1)
        y2 = y1.view(-1,nt,filters)   ####   ET       
        y1 = self.conv2(y1)
        y1 = self.conv3(y1)
        #y1 = self.leaky(y1)
        y3 = y1.view(-1,nt,filters)   ####   ETBAR
        
        z1 = self.fc3(z)    ####  DT
        z2 = F.linear(z1, self.fc3.weight.t())    #### DTBAR

        
        w = torch.stack((x, x2,z, z2), dim=2)
        v = torch.cat((y2,y3),dim =2)
        u = torch.cat((w,v),dim=2)
        a = a.view(-1,nt,1)
        r = torch.cat((a,u),dim = 2)
        
        #print(r.shape)
        r = r.view(-1,1,nt,5+2*(filters))
        #t = self.conv4(r)
        #s = t.view(-1,nt)
        t = self.fc4(r)
        s = t.view(-1,nt)
        #s = torch.diagonal(t, dim1=1, dim2=2)
        #print(s.shape)

        return s

In [ ]:
torch.ones([32, 50], dtype=torch.float64).shape

In [ ]:
model = DNN_f()
model = model.to(device)

In [ ]:

# custom function
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

# define vectorized sigmoid
sigmoid_v = np.vectorize(sigmoid)


In [ ]:
optimizer = Adam(model.parameters(), lr=5e-3)
loss_func = nn.BCEWithLogitsLoss(reduction = 'mean').cuda()

BATCH_SIZE = 32
train_loss = 0
losses = []
steps = []
step = 0
count = 0
EPOCHS = 50
trained = []
tested = []

In [ ]:
X_indicator = torch.ones([107017, nt])

In [ ]:

import time
start_time = time.time()
train_dataset = TensorDataset(X_freq, X_recent, X_recommend, y, X_indicator)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)
print ("--- %s seconds ---" % (time.time() - start_time))


for epoch_num in range(EPOCHS):
    model.train()
    for step_num, batch_data in enumerate(train_dataloader):
        freq, recent, recommend, labels, indicator = tuple(t.to(device)for t in batch_data)
        probas = model(freq, recent, recommend, indicator)
        batch_loss = loss_func(probas, labels)
        train_loss += batch_loss.item()

        model.zero_grad()
        batch_loss.backward()


        #clip_grad_norm_(parameters=model.parameters(), max_norm=1.0)
        optimizer.step()

        clear_output(wait=True)
        print('Epoch: ', epoch_num + 1)
        print("{0}/{1} loss: {2} ".format(step_num, len(y)/BATCH_SIZE, train_loss / (count + 1)))
        print ("--- %s seconds ---" % (time.time() - start_time))
        losses.append(batch_loss.item())
        steps.append(step)
        step += 1
        count += 1
#     lg_train = model(X_freq.to(device), X_recent.to(device), X_recommend.to(device), X_indicator.to(device)).to('cpu')
#     numpy_lg_train = lg_train.detach().numpy()
#     prob_tr = sigmoid_v(numpy_lg_train)
#     trained.append(log_loss(y.numpy(),prob_tr))
    lg_test = model(X_freq_test.to(device), X_recent_test.to(device), X_recommend_test.to(device), X_indicator.to(device)).to('cpu')
    numpy_lg_test = lg_test.detach().numpy()
    prob_tt = sigmoid_v(numpy_lg_test)
    tested.append(log_loss(Y_test.numpy(),prob_tt))


In [ ]:


#torch.save({'state_dict': model.state_dict()}, 'checkpoint.pth.tar')

#model = model()
#checkpoint = torch.load('checkpoint.pth.tar')
#model.load_state_dict(checkpoint['state_dict'])

torch.save(model, "model_dnn_july19_10l_20f.pkl")

In [ ]:
#model = torch.load("model_dnn_march30_12743.pkl")
model = model.to(device)

In [ ]:
model.eval()

with torch.no_grad():

    logits_train = model(X_freq.to(device), X_recent.to(device), X_recommend.to(device), X_indicator.to(device)).to('cpu')
    logits_test = model(X_freq_test.to(device), X_recent_test.to(device), X_recommend_test.to(device), X_indicator.to(device)).to('cpu')
    numpy_logits_train = logits_train.detach().numpy()
    numpy_logits_test = logits_test.detach().numpy()
    
        

In [ ]:
#Converting logits to probabilities
numpy_probas_test = sigmoid_v(numpy_logits_test)
numpy_probas_train = sigmoid_v(numpy_logits_train)

In [ ]:
from sklearn.metrics import log_loss
log_loss(Y_test.numpy(),numpy_probas_test)

In [ ]:
#Performance Metrics          
def binary_classification_performance(y_test, y_pred):
    tp, fp, fn, tn = confusion_matrix(y_test, y_pred).ravel()
    accuracy = round(accuracy_score(y_pred = y_pred, y_true = y_test),2)
    precision = round(precision_score(y_pred = y_pred, y_true = y_test),2)
    recall = round(recall_score(y_pred = y_pred, y_true = y_test),2)
    f1_score = round(2*precision*recall/(precision + recall),2)
    specificity = round(tn/(tn+fp),2)
    npv = round(tn/(tn+fn),2)


    result = pd.DataFrame({'Accuracy' : [accuracy],
                         'Precision (or PPV)' : [precision],
                         'Recall (senitivity or TPR)' : [recall],
                         'f1 score' : [f1_score],
                         'Specificty (or TNR)': [specificity],
                         'NPV' : [npv],
                         'True Positive' : [tp],
                         'True Negative' : [tn],
                         'False Positive':[fp],
                         'False Negative':[fn]})
    return result





In [ ]:
thresh = []
roc = []
for i in range(nt):    
    import sklearn.metrics as metrics
    fpr, tpr, threshold = metrics.roc_curve(Y_test[:,i].numpy(), numpy_probas_test[:,i])
    roc_auc = metrics.auc(fpr, tpr)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = threshold[optimal_idx]
    roc.append(roc_auc)
    thresh.append(optimal_threshold)
    
    gmeans = np.sqrt(tpr * (1-fpr))
    ix = np.argmax(gmeans)
    print('Best Threshold=%f, G-Mean=%.3f' % (threshold[ix], gmeans[ix]))

    # method I: plt
    import matplotlib.pyplot as plt
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.scatter(fpr[ix], tpr[ix], marker='o', color='black', label='Best')
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

In [ ]:
sum(roc)/nt

In [ ]:
def to_labels(pos_probs, threshold):
	return (pos_probs >= threshold).astype('int')

for i in range(nt):
    numpy_probas_test[:,i] = to_labels(numpy_probas_test[:,i],thresh[i])

In [ ]:
scores = []
for i in range(nt):
    scores.append(binary_classification_performance(Y_test[:,i].numpy(),numpy_probas_test[:,i]))

In [ ]:
f = pd.concat(scores)

In [ ]:
f.mean(axis=0)

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(Y_test.numpy(), numpy_probas_test)

# MNL model

In [ ]:
before_norm_df = history_frequency_train.loc[:, history_frequency_train.columns != 'engaging_user_id']
before_norm_df.loc['Total',:]= before_norm_df.sum(axis=0)
before_norm_df.loc[0:107016,:] = before_norm_df.loc['Total'].values

In [ ]:
before_norm_df.drop(['Total'],inplace=True)
before_norm_df

In [ ]:
after_norm_df = before_norm_df/1890150


In [ ]:
recom = recommend_test.drop('engaging_user_id', axis=1)

In [ ]:
df3 = np.multiply(after_norm_df, recom)

In [ ]:
df3['50'] = df3.sum(axis=1)

In [ ]:
df3['50'] = df3['50']+1156967/1890150

In [ ]:
df_new = df3.loc[:,"0":"49"].div(df3["50"], axis=0)

In [ ]:
prob_mnl = df_new.to_numpy()

In [ ]:
from sklearn.metrics import log_loss
log_loss(Y_test.numpy(),prob_mnl)

In [ ]:
thresh = []
roc = []
for i in range(nt):    
    import sklearn.metrics as metrics
    fpr, tpr, threshold = metrics.roc_curve(Y_test[:,i].numpy(), prob_mnl[:,i])
    roc_auc = metrics.auc(fpr, tpr)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = threshold[optimal_idx]
    roc.append(roc_auc)
    thresh.append(optimal_threshold)
    
    gmeans = np.sqrt(tpr * (1-fpr))
    ix = np.argmax(gmeans)
    print('Best Threshold=%f, G-Mean=%.3f' % (threshold[ix], gmeans[ix]))

    # method I: plt
    import matplotlib.pyplot as plt
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.scatter(fpr[ix], tpr[ix], marker='o', color='black', label='Best')
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

# UCM Model

In [ ]:
# data = read_data("data.txt")
# choices_to_add = read_data_choices_to_add("freqent_itemset_size2.txt")
# ucm_tafeng_H_twitter = initialize_model_with_H(data, choices_to_add)

# a = log_likelihood(ucm_tafeng_H_twitter, data)

# b = get_subset_counts(data)

# Binary Logits

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [ ]:
#Converting engagement frequencies from tensor to numpy array

X_freq_1 = X_freq.numpy()
X_freq_1_test = X_freq_test.numpy()


In [ ]:
empty_train_p = []
empty_test_p = []
for i in range(nt):
    recent_lm_train = X_recent_2[:,[i,i+1*nt,i+2*nt,i+3*nt]]     #Engagement history by topic as input for training
    freq_lm_train = X_freq_1[:,i]                            #Engagement frequncy by topic as input for training
    recommend_lm_train = X_recommend_1[:,i]                  #Recommendations by topic as input for training
    
    recent_lm_test = X_recent_2_test[:,[i,i+1*nt,i+2*nt,i+3*nt]] #Engagement history by topic as input for testing
    freq_lm_test = X_freq_1_test[:,i]                        #Engagement frequncy by topic as input for testing
    recommend_lm_test = X_recommend_t[:,i]                   #Recommendations by topic as input for testing


    y_lm_train = y_1[:,i]                                    #Engagement labels

    X = np.column_stack([recent_lm_train,freq_lm_train,recommend_lm_train])  #Concatenated input features for training

    #LR = LogisticRegression(class_weight="balanced")
    LR = LogisticRegression()

    LR.fit(X,y_lm_train)
    
    X_test = np.column_stack([recent_lm_test,freq_lm_test,recommend_lm_test]) #Concatenated input features for testing
    
    empty_train_p.append((LR.predict_proba(X)[:,1]))
       
    empty_test_p.append((LR.predict_proba(X_test)[:,1]))
      

In [ ]:
prediction_lr_train_p = np.transpose(np.array(empty_train_p))  #Predictions for training data
prediction_lr_test_p = np.transpose(np.array(empty_test_p))    #Predictions for test data

In [ ]:
from sklearn.metrics import log_loss
log_loss(y_t,prediction_lr_test_p)

In [ ]:
thresh_lr = []
roc_lr = []
for i in range(nt):    
    import sklearn.metrics as metrics
    fpr, tpr, threshold = metrics.roc_curve(Y_test[:,i].numpy(), prediction_lr_test_p[:,i])
    roc_auc = metrics.auc(fpr, tpr)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = threshold[optimal_idx]
    thresh_lr.append(optimal_threshold)
    roc_lr.append(roc_auc)
    
    gmeans = np.sqrt(tpr * (1-fpr))
    ix = np.argmax(gmeans)
    print('Best Threshold=%f, G-Mean=%.3f' % (threshold[ix], gmeans[ix]))

    # method I: plt
    import matplotlib.pyplot as plt
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.scatter(fpr[ix], tpr[ix], marker='o', color='black', label='Best')
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show() 

In [ ]:
sum(roc_lr)/nt  #Average AUC-ROC score 

In [ ]:
prediction_lr_test_p.shape

In [ ]:
label_lr_test_p = np.empty([107017, nt])
for i in range(nt):
    label_lr_test_p[:,i] = to_labels(prediction_lr_test_p[:,i],thresh_lr[i])

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(Y_test.numpy(), label_lr_test_p)


In [ ]:
scores_lr = []
for i in range(nt):
    scores_lr.append(binary_classification_performance(Y_test[:,i].numpy(),label_lr_test_p[:,i]))

In [ ]:
f_lr = pd.concat(scores_lr)

In [ ]:
f_lr.mean(axis=0)

# Light GBM model

In [ ]:
!pip install lightgbm

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier



In [ ]:
empty_train_1p = []
empty_test_1p = []


for i in range(nt):
    recent_lm_train = X_recent_2[:,[i,i+nt,i+2*nt,i+3*nt]]
    freq_lm_train = X_freq_1[:,i]
    recommend_lm_train = X_recommend_1[:,i]
    
    recent_lm_test = X_recent_2_test[:,[i,i+nt,i+2*nt,i+3*nt]]
    freq_lm_test = X_freq_1_test[:,i]
    recommend_lm_test = X_recommend_t[:,i]


    y_lm_train = y_1[:,i]
    

    X = np.column_stack([recent_lm_train,freq_lm_train,recommend_lm_train])

    LGBM = LGBMClassifier()

    LGBM.fit(X,y_lm_train)
    
    X_test = np.column_stack([recent_lm_test,freq_lm_test,recommend_lm_test])
    
    empty_train_1p.append((LGBM.predict_proba(X)[:,1]))

    empty_test_1p.append((LGBM.predict_proba(X_test)[:,1]))
        


In [ ]:
prediction_lgbm_train_p = np.transpose(np.array(empty_train_1p))
prediction_lgbm_test_p = np.transpose(np.array(empty_test_1p))

In [ ]:
from sklearn.metrics import log_loss
log_loss(Y_test.numpy(),prediction_lgbm_test_p)

In [ ]:
thresh_lgbm = []
roc_lgbm = []
for i in range(nt):    
    import sklearn.metrics as metrics
    # calculate the fpr and tpr for all thresholds of the classification
    #probs = model.predict_proba(X_test)
    #preds = probs[:,1]
    fpr, tpr, threshold = metrics.roc_curve(Y_test[:,i].numpy(), prediction_lgbm_test_p[:,i])
    roc_auc = metrics.auc(fpr, tpr)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = threshold[optimal_idx]
    print(optimal_threshold)
    thresh_lgbm.append(optimal_threshold)
    roc_lgbm.append(roc_auc)
    
    gmeans = np.sqrt(tpr * (1-fpr))
    ix = np.argmax(gmeans)
    print('Best Threshold=%f, G-Mean=%.3f' % (threshold[ix], gmeans[ix]))

    # method I: plt
    import matplotlib.pyplot as plt
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.scatter(fpr[ix], tpr[ix], marker='o', color='black', label='Best')
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show() 

In [ ]:
sum(roc_lgbm)/nt

In [ ]:
label_lgbm_test_p = np.empty([107017, nt])
for i in range(nt):
    label_lgbm_test_p[:,i] = to_labels(prediction_lgbm_test_p[:,i],thresh_lgbm[i])

In [ ]:
multilabel_confusion_matrix(Y_test.numpy(), label_lgbm_test_p)

In [ ]:
scores_lgbm = []
for i in range(nt):
    scores_lgbm.append(binary_classification_performance(Y_test[:,i].numpy(),label_lgbm_test_p[:,i]))

In [ ]:
f_lgbm = pd.concat(scores_lgbm)

In [ ]:
f_lgbm.mean(axis=0)

# Exhaustive search for Optimization

In [ ]:
x = pd.DataFrame(X_recommend_t)
x.sum(axis=1).value_counts()
x['Total'] = x.sum(axis=1)
y=x[x['Total'] >= 6] #Considering only the users who are shown atleast six tweets
#del y['Total']

infinity = pd.DataFrame(X_freq_1_test)
histo = pd.DataFrame(X_recent_2_test)

inf_new=infinity.loc[y.index,:]
hist_new=histo.loc[y.index,:]

# Greedy Search

In [ ]:



def greedy(freq, hist, y, nt):
    empty =[]
    for user in range(y.shape[0]):
        s=set()
        for i,x in enumerate(y.iloc[user]):
            if x==1.0:
                s.add(i)
        output=[[0.0]*nt]
        removed=set()
        l_=[]

        for j in range(5):
            ma=-float('inf')
            for i in s:
                l_.append(i)
                output[0][i]=1.0
                inf_new_ = torch.FloatTensor(freq.iloc[user].values).reshape(1,nt)
                hist_new_ = torch.FloatTensor(hist.iloc[user].values.reshape(-1,nt,4))
                y_ = torch.FloatTensor(np.array(output[0])).reshape(1,nt)
                indicator = torch.ones([nt]).reshape(1,nt)
                
                logits_opt = model(inf_new_.to(device), hist_new_.to(device), y_.to(device), indicator.to(device))
                numpy_logits_opt = logits_opt.to('cpu').detach().numpy()
                numpy_probas_opt = sigmoid_v(numpy_logits_opt)
                sum_=sum(numpy_probas_opt[0][k] for k in l_)
                if sum_>ma:
                    result_l=l_[:]
                    numpy_logits_opt_max=numpy_logits_opt
                    numpy_probas_opt_max=numpy_probas_opt
                    ma=sum_
                    index=i
                l_.pop()
                output[0][i]=0.0

            output[0][index]=1.0
            l_.append(index)

            s.remove(index)
        empty.append([y.index[user],ma,result_l])
    n=pd.DataFrame(empty,columns=['USER','MAX_SUM','LIST_OF_INDEX'])
    return (n['MAX_SUM'].mean())
        

In [ ]:
greedy(inf_new, hist_new, y, nt)

# Uplift score obtained using combination of topics a given model

In [ ]:
#prob : output probabilities for a given model
#y : input recommendations considering overall timeline
#freq: engagement frequency
#hist: engagement history

def uplift_score(prob, freq, hist, y, nt):     
    mdl = pd.DataFrame(prob)
    mdl_y = mdl.loc[y.index]
    p = mdl_y*y.iloc[:,0:nt].values
    s=pd.DataFrame(np.zeros((p.shape[0],p.shape[1]))).set_index(p.index)
    for ind in p.index:
        s.loc[ind,p.loc[ind].nlargest(5).index.values]=1
    s_=s.reset_index()
    s_.index = s_['index']
    del s_['index']
    inf_new_T = torch.FloatTensor(freq.values)
    hist_new_T = torch.FloatTensor(hist.values.reshape(-1,nt,4))
    rec_T = torch.FloatTensor(s_.values)
    indicator_T = torch.ones([len(rec_T), nt])
    logits_opt = model(inf_new_T.to(device), hist_new_T.to(device), rec_T.to(device), indicator_T.to(device))
    numpy_logits_opt = logits_opt.to('cpu').detach().numpy()
    numpy_probas_opt = sigmoid_v(numpy_logits_opt)
    sp = pd.DataFrame(numpy_probas_opt, index = y.index)
    sc = sp*s_.values
    return(sc.sum(axis = 1).mean())

In [ ]:
uplift_score(prediction_lr_test_p,inf_new, hist_new, y, nt)

In [ ]:
uplift_score(prediction_lgbm_test_p,inf_new, hist_new, y, nt)